### Imports

In [ ]:
from tfkld import TFKLD
import pandas as pd
from scipy.sparse import vstack, lil_matrix
from cPickle import load, dump
from scipy.sparse.linalg import svds
import numpy, gzip
import cPickle as pickle
import os
import numpy as np
from sklearn.externals import joblib

### Helper Functions

This code was pulled from this git repo https://github.com/smujjiga/tfkld I've included the fe_quora.py file in this repository

In [ ]:
class DimReduction(object):
    def __init__(self, M, K):
        """ Initialization
        """
        self.M = M
        self.K = K


    def svd(self):
        U, s, Vt = svds(self.M, k=self.K)
        W = U.dot(numpy.diag(s))
        H = Vt
        return (W, H)


    def nmf(self):
        pass


def dr(trainX, devX, testX, K=200):
    n_train = trainX.shape[0]
    n_dev = devX.shape[0]
    n_test = testX.shape[0]

    M = vstack([trainX, devX, testX])
    print 'M.shape = {}'.format(M.shape)
    dr = DimReduction(M, K)
    W, H = dr.svd()
    # Split data
    trainX = W[:n_train, :]
    devX = W[n_train:n_train+n_dev, :]
    testX = W[n_train+n_dev:, :]

    return trainX, devX, testX


def split_train(train):
    if os.path.exists("dev_data.csv"):
        return 

    print ("Splitting train data into train and dev sets")
    df = pd.read_csv(train)
    n = len(df)
    # Shuffle and split
    df = df.iloc[np.random.permutation(n)]
    per = .75
    df[:int(per*n)].to_csv("train_data.csv", index=False)
    df[int(per*n):].to_csv("dev_data.csv", index=False)

### Training Code

In [ ]:
split_train("train.csv")
    
TRAIN_DATA = "train_data.csv"
DEV_DATA = "dev_data.csv"
TEST_DATA = "test.csv"

print "Loading Train data"

In [ ]:
train_df = pd.read_csv(TRAIN_DATA, usecols=['question1', 'question2', 'is_duplicate'])
train_df = train_df.fillna(' ')
trainY = train_df['is_duplicate']
trainX = list()
for i, row in train_df.iterrows():
    trainX.append(row['question1'])
    trainX.append(row['question2'])
print ("Train: {0}".format(train_df.shape))
del train_df

In [ ]:
tfkld = TFKLD()
print ("Fitting the model")
tfkld.fit(trainX, trainY)

### Save Model

In [ ]:
pickle.dump(tfkld, open("tfkld.pkl", "wb"))

### Test Model

In [ ]:
my_vec = tfkld.transform(["Hello what is your name?"])
print my_vec

### Load and use Model

In [ ]:
my_file = open("tfkld.pkl", "r")
tfkld_model = pickle.load(my_file)
my_file.close()

In [ ]:
my_vec = tfkld_model.transform(["test this sentence"])
print my_vec[0]
print "\n next vec \n"
# print my_vec[1]